# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [26]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
dados_rj = requests.get(url)  
dados_rj.status_code
data_rj_json = dados_rj.json()
df_rj = pd.DataFrame(data_rj_json['items'])
df_rj.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,6660,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.000403/2010-81,2010-05-24T03:00:00Z,Operação contratual interna,PAC - Favelas,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.060466e+08,1,0,06/07/2010
1,9050,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000893/2010-16,2011-06-03T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.591383e+08,1,0,04/07/2011


In [28]:
# 2) Seu código aqui
df_rj['status'].value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: count, dtype: int64

In [34]:
# 3) Seu código aqui
ano_data_status = df_rj['data_status']
ano_status = ano_data_status.str.slice(start=6, stop=10, step=1)


In [36]:
#4 
ano_status.value_counts()

data_status
2012    11
2009     9
2013     7
2011     6
2016     4
2002     4
2014     4
2010     3
2008     2
2006     2
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [52]:
#1) Seu código aqui

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?tipo_interessado=Estado'
dados_uf = requests.get(url)  
#dados_uf.status_code
data_uf_json = dados_uf.json()

df_uf = pd.DataFrame(data_uf_json['items'])
df_uf.head(2)
print('Digite o estado (UF) desejado')
filtro_df_uf = df_uf.loc[df_uf['uf'] == input().upper()]
filtro_df_uf.head()

Digite o estado (UF) desejado


 sp


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
3,5248,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000068/2013-64,2014-07-01T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,2.000000e+08,1,0,07/08/2014
11,5701,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000175/2013-92,2013-06-13T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.000000e+08,1,0,03/07/2013
14,67164,Estado,São Paulo,35,SP,PVL02.003925/2023-97,Em análise,17944.105321/2023-47,2023-11-10T19:30:44Z,Operação contratual interna (com garantia da U...,Aporte em PPP,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.200000e+09,0,0,16/11/2023
35,9123,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000908/2005-89,2005-07-28T03:00:00Z,Operação contratual externa (com garantia da U...,Preservação e recuperação ambiental,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,9.000000e+06,1,0,25/01/2006
61,7476,Estado,São Paulo,35,SP,None,Deferido,17944.000588/2014-58,2014-04-16T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.700000e+09,1,0,30/04/2014


In [54]:
# 2) Seu código aqui

df_mg = df_uf.loc[df_uf['uf'] == 'MG']
df_mg['status'].value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: count, dtype: int64

In [56]:
# 3) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio'
mun_ba = requests.get(url)  
data_uf_json = mun_ba.json()

df_ba_mun = pd.DataFrame(data_uf_json['items'])
df_ba_mun.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,60731,Município,Barra da Estiva,2902807,BA,PVL02.007122/2022-21,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,4000000.0,0,1,15/07/2022
1,64401,Município,Barreiras,2903201,BA,PVL02.001147/2023-00,Deferido,17944.101999/2023-51,2023-05-10T13:55:39Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,40000000.0,0,1,17/05/2023


In [68]:
df_ba_def = df_ba_mun.loc[df_ba_mun['status'] == 'Deferido']
df_ba_def['interessado'].value_counts()

interessado
Luís Eduardo Magalhães    16
Vitória da Conquista      12
Barreiras                 10
Camaçari                  10
Lauro de Freitas          10
                          ..
Malhada                    1
Sobradinho                 1
Cafarnaum                  1
Itapicuru                  1
Itagibá                    1
Name: count, Length: 185, dtype: int64

In [70]:
# 4) Seu código aqui
df_ba_def.to_csv('dados_estado_Bahia')